In [1]:
from getpass import getpass
import os, sys, argparse, paramiko, re

# Create password prompt 

thePass = getpass(prompt="Enter Password for SSH: ")

# Host information 

host = "192.168.6.71"
port = 2222
username = "ciaran.byrne"
password = thePass 

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host,port,username,password)

except paramiko.AuthenticationException():
    print("Authentication Failed")


In [20]:
# Kraken analysis 
sftp = ssh.open_sftp()

sftp.put('kraken','/home/ciaran.byrne/kraken')

com = 'chmod 777 ./kraken'
stdin, stdout, stderr = ssh.exec_command(com)


com = 'sudo -S ./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin '
stdin, stdout, stderr = ssh.exec_command(com)

stdin.write(password + "\n")
stdin = 0
if stderr.channel.recv_exit_status() != 0:
    print('The Following Error occured: {}'.format(stderr.readlines()))
    print()


for line in iter(stdout.readline,""):
    print(line,end="")

INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0005] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0017] Scanning autoruns...                         
WARN[0017] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0018] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0018] Scanning the filesystem (this can take several minutes)... 
ERRO[0018] Some malicious artifacts have been detected on this system: 
ERRO[0018] Found detection for foundGoBinary            
ERRO[0018] Found detection for foundGoBinary            
ERRO[0018] Found detection for foundGoBinary      

In [21]:
# PID searching 


processes = []

while True:
    pid = input("Please Enter Suspicious PID: ")
    processes.append(pid)
    con = input("Would you like to Enter more PIDs? (Yes or No): ")
    if con == "No":
        break

results = []
i = 0
for eachProc in processes: 
    com = 'sudo -S lsof -p {}'.format(processes[i])
    stdin, stdout, stderr = ssh.exec_command(com)

    stdin.write(password + "\n")
    stdin.flush()

    results.append(stdout.readlines())
    i = i + 1 
i = 0 
for eachRes in results:
    print("###### Results for PID {} ######\n\n".format(processes[i]))
    for line in eachRes:
        print(line,end="")
    i = i + 1 
    print('\n\n')

###### Results for PID 3138  ######


COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME
ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk
ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /
ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls
ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so
ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    4u      REG    8,1        4 

In [7]:
# Download Suspicious Files

def get_file(file):
    sftp = ssh.open_sftp()

    filename = file.split('/')
    
    target_path = './Suspicious/' + filename[-1]
    
    sftp.get(file,target_path)

get_file("/home/p.dalton/test-lk/.d.log")
get_file("/home/p.dalton/test-lk/d.pid")
get_file("/usr/local/bin/ls")

In [5]:
# Task 5 

def run_command(comm):
    stdin,stdout,stderr = ssh.exec_command("sudo -S " + comm)
    stdin.write(password + "\n")
    stdin.flush()

    for line in iter(stdout.readline,""):
        print(line,end="")

def get_log(file):
    sftp = ssh.open_sftp()

    filename = file.split('/')

    target_path = './logs/' + filename[-1]

    sftp.get(file,target_path)



print("\n\n### Processes on PID 3138 ###\n\n")

command = "lsof -p 3138 | tee pid.txt"
run_command(command)


print("\n\n### Looking for suspicious account names with naming convention ###\n\n")

command = "cat /etc/passwd | grep '^[a-z]\.[a-z]*' | tee passwd.txt"
run_command(command)

print("\n\n### Searching Audit log for suspicious naming convention ###\n\n")

command = "cat /var/log/auth.log | grep p.dalton | tee auth.txt"
run_command(command)

print("\n\n### last login for p.dalton ###\n\n")

command = "last | grep p.dalton | tee last.txt"
run_command(command)

print("\n\n### Copying and downloading suspicious user's directory ###\n\n")
command = 'cp /home/p.dalton/test-1k ./'
command = 'ls test-1k'



get_log("passwd.txt")
get_log("auth.txt")
get_log("pid.txt")
get_log("last.txt")



### Processes on PID 3138 ###


COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME
ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk
ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /
ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls
ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so
ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    4u      REG    8,1        4 2581

In [ ]:
ssh.close()

# Task 5 Answers

#### What is the process name?
#### The process was ls 

#### Which user is running the process?
#### p.dalton 

#### Has the user logged in before via SSH?
#### Authentication logs do not date back before Sunday March 6th 2022 so not since then. 

#### When did the user login?
#### Logs have been cleared but the last time p.dalton was logged in was Feb 27th however was last SSH'd into the system on Feb 26th 

#### How long was the login session
#### 24 minutes locally and 36 minutes remotely

#### What is located in the unauthorized user’s home directory?
#### .d.log and d.log 

#### Download any suspicious files.
#### Done. 


# Reflections 

#### Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?

#### What i can see the unauthorised user logged in using ssh then started a service enabled on the ls command 
